In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time

In [2]:
years = [2023,2022,2021,2020]
total_list = []
laliga_url = 'https://fbref.com/en/comps/12/La-Liga-Stats'

In [3]:
for year in years:
    data = requests.get(laliga_url)
    html = data.text #take html data
    bs = BS(html)
    time.sleep(5) #pause is crusial, because without it site with data would not let parsing
    total_table = bs.select('table.stats_table')[0]

    hyperlinks = [link.get('href') for link in total_table.find_all('a')] #getting link for every element that has 'a'- flag of hyperlink
    hyperlinks = [link for link in hyperlinks if '/squads/' in link]
    
    teams_url = [f'https://fbref.com{link}' for link in hyperlinks] #add url of every team in championship
    
    season_before = bs.select('a.prev')[0].get('href') #get link for next iteration (previous season)
    laliga_url = f'https://fbref.com{season_before}'
    
    for team in teams_url:
        team_name = team.split('/')[-1].replace('-Stats', '')
        data = requests.get(team)
        html = data.text
        matches = pd.read_html(html, match="Scores & Fixtures")[0]
        
        bs = BS(html)
        hyperlinks = [link.get('href') for link in bs.find_all('a')]
        hyperlinks = [link for link in hyperlinks if link and 'all_comps/shooting/' in link]
        
        data = requests.get(f'https://fbref.com{hyperlinks[0]}')
        html = data.text
        shooting = pd.read_html(html, match='Shooting')[0] #get data from other table with information about goals, shootings, etc.
        shooting.columns = shooting.columns.droplevel()
                            
        try:
            team_data = matches.merge(shooting[['Date','Sh', 'SoT','G/Sh','G/SoT', 'Dist','PK', 'FK', 'PKatt']], on='Date')
        except ValueError: #in case of absense of some values
            continue
            
        team_data = team_data[team_data['Comp'] == 'La Liga'] #leave only La Liga championship
        team_data['Season'] = f'{year-1}-{year}'
        team_data['Team'] = team_name
        
        total_list.append(team_data)
        time.sleep(5)

In [4]:
df = pd.concat(all_matches)

NameError: name 'all_matches' is not defined

In [ ]:
df.to_csv('matches_laliga.csv')

In [ ]:
df